In [1]:
import pandas as pd
songs = pd.read_csv("../data/songs.csv")
streams = pd.read_csv("../data/streams1.csv")

ModuleNotFoundError: No module named 'pandas'

In [2]:
songs.head()

,id,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [3]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89741 entries, 0 to 89740
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                89741 non-null  int64  
 1   track_id          89741 non-null  object 
 2   artists           89740 non-null  object 
 3   album_name        89740 non-null  object 
 4   track_name        89740 non-null  object 
 5   popularity        89741 non-null  int64  
 6   duration_ms       89741 non-null  int64  
 7   explicit          89741 non-null  bool   
 8   danceability      89741 non-null  float64
 9   energy            89741 non-null  float64
 10  key               89741 non-null  int64  
 11  loudness          89741 non-null  float64
 12  mode              89741 non-null  int64  
 13  speechiness       89741 non-null  float64
 14  acousticness      89741 non-null  float64
 15  instrumentalness  89741 non-null  float64
 16  liveness          89741 non-null  float6

In [6]:
songs['key'].head()

0    1
1    1
2    0
3    0
4    2
Name: key, dtype: int64

In [8]:
streams.head()

,user_id,track_id,listen_time
0,26213,4dBa8T7oDV9WvGr7kVS4Ez,2024-06-25 17:43:13
1,6937,4osgfFTICMkcGbbigdsa53,2024-06-25 07:26:00
2,21407,2LoQWx41KeqOrSFra089YS,2024-06-25 13:25:26
3,47146,7cfG5lFeJWEgpSnubt4O4W,2024-06-25 18:17:50
4,38594,6tilCYbheGMHo3Hw4F22hF,2024-06-25 17:33:21


In [9]:
streams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11346 entries, 0 to 11345
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      11346 non-null  int64 
 1   track_id     11346 non-null  object
 2   listen_time  11346 non-null  object
dtypes: int64(1), object(2)
memory usage: 266.1+ KB


In [23]:
# Prepare the data
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
merged_data = streams.merge(songs, on='track_id', how='left')

# Compute each KPI
# KPI 1: Daily Genre Listen Count
genre_listen_count = merged_data.groupby(['listen_date', 'track_genre']).size().reset_index(name='listen_count')


# KPI 2: Average Listening Duration per Genre per Day
merged_data['duration_seconds'] = merged_data['duration_ms'] / 1000
avg_duration = merged_data.groupby(['listen_date', 'track_genre'])['duration_seconds'].mean().reset_index(name='average_duration')

# KPI 3: Daily Genre Popularity Index
total_listens = merged_data.groupby('listen_date').size().reset_index(name='total_listens')
genre_listen_count = genre_listen_count.merge(total_listens, on='listen_date')
genre_listen_count['popularity_index'] = genre_listen_count['listen_count'] / genre_listen_count['total_listens']

# KPI 4: Most Popular Track
most_popular_track = merged_data.groupby(['listen_date', 'track_genre', 'track_id']).size().reset_index(name='track_count')
most_popular_track = most_popular_track.sort_values(by=['listen_date', 'track_genre', 'track_count'], ascending=[True, True, False])
most_popular_track = most_popular_track.drop_duplicates(subset=['listen_date', 'track_genre'], keep='first').rename(columns={'track_id': 'most_popular_track_id'})

# Combine all the KPIs into a single data frame
final_kpis = genre_listen_count[['listen_date', 'track_genre', 'listen_count', 'popularity_index']]
final_kpis = final_kpis.merge(avg_duration, on=['listen_date', 'track_genre'])
final_kpis = final_kpis.merge(most_popular_track[['listen_date', 'track_genre', 'most_popular_track_id']], on=['listen_date', 'track_genre'])

In [24]:
final_kpis.head()

,listen_date,track_genre,listen_count,popularity_index,average_duration,most_popular_track_id
0,2024-06-25,acoustic,118,0.010400,215.379034,0WWuB1F1H4dr3Bdoe1vtHs
1,2024-06-25,afrobeat,124,0.010929,248.978960,1KME77F9mu2RQS8vo6JVwa
2,2024-06-25,alt-rock,119,0.010488,236.073731,0GO8y8jQk1PkHzS31d699N
3,2024-06-25,alternative,45,0.003966,202.768200,0YwBrYaPYYc8e18ZYkqhJc
4,2024-06-25,ambient,119,0.010488,255.296067,2tr4oclswJ6v3dfDlI01HD


In [25]:
final_kpis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   listen_date            113 non-null    object 
 1   track_genre            113 non-null    object 
 2   listen_count           113 non-null    int64  
 3   popularity_index       113 non-null    float64
 4   average_duration       113 non-null    float64
 5   most_popular_track_id  113 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 5.4+ KB
